### 1. Imports

In [ ]:
from pathlib import Path
import json, numpy as np, matplotlib.pyplot as plt

from utils.config import PART_COLORS
from utils.mask_utils import load_mask
from utils.voxel_utils import get_voxel_points_by_parts
from utils.camera_geometry import project
from utils.projection_utils import visualize_reprojection, project_colored_voxels
from utils.visualization import plot_voxel
from utils.camera_estimation import *

### 2. Load data

In [ ]:
root = Path.cwd()
data = root / "data"
monuments = ["Akbar","Bibi","Charminar","Itimad","Taj"]
monument = monuments[1] #choose any other
voxel_grid = np.load(root/"results/1.Orthographic_Voxel_Carving"/f"{monument}_voxel_grid.npz")["voxel_grid"] # or choose results_temp if you ran notebook 1
max_dim = np.max(voxel_grid.shape)

front_mask = load_mask(data, monument, "front", max_dim)
drone_mask = load_mask(data, monument, "drone")
print("Voxel grid:", voxel_grid.shape,
      "| Front mask:", front_mask.shape,
      "| Drone mask:", drone_mask.shape)
for i,(t,m) in enumerate([("Front",front_mask),("Drone",drone_mask)]):
    plt.subplot(1,2,i+1); plt.imshow(m); plt.title(t); plt.axis("off")
plt.tight_layout(); plt.show()


### 3. Initialize camera

In [ ]:
# ---------- INITIAL CAMERA PARAMS (ALL VIEWS) ----------

views = {
    "front":  front_mask,
    "drone":  drone_mask,
}

parts_for_alignment = ["front_minarets", "back_minarets"]
minaret_colors = [PART_COLORS["front_minarets"], PART_COLORS["back_minarets"]]

init_camera_params, init_keypoints = {}, {}

for view, mask_img in views.items():
    print(f"\n📐 {view}")

    try:
        voxel_kps, image_kps = extract_minaret_kps_for_view(
            voxel_grid, mask_img, minaret_colors
        )

        init_params = auto_compute_initial_params_matching_bbox(
            voxel_grid,
            mask_img,
            PART_COLORS,
            parts_for_alignment=parts_for_alignment,
            fov_deg=30,
        )

        visualize_reprojection(
            mask_img,
            voxel_kps,
            image_kps,
            init_params,
            title=f"{view.capitalize()} | Initial Reprojection",
        )

        init_camera_params[view] = init_params
        init_keypoints[view] = {
            "voxel": voxel_kps,
            "image": image_kps,
            "mask":  mask_img,
        }

    except Exception as e:
        print(f"⚠️ skipped: {e}")


### 4. Keypoint based optimization

In [ ]:
kp_camera_params = {}

for view_name, data in init_keypoints.items():
    print(f"\n🎯 Optimizing camera for view: {view_name}")

    init_params = init_camera_params[view_name]
    voxel_kps_sel = data["voxel"]
    image_kps_sel = data["image"]
    mask_img = data["mask"]

    final_params = optimize_camera_with_keypoints(
        voxel_keypoints_dict=voxel_kps_sel,
        image_keypoints_dict=image_kps_sel,
        image=mask_img,
        init_params=init_params,
        loss_type='L2'
    )

    kp_camera_params[view_name] = final_params

    visualize_reprojection(
        mask_img,
        voxel_kps_sel,
        image_kps_sel,
        final_params,
        title=f"{view_name.capitalize()} View: Final Reprojection"
    )

    visualize_voxel_projection_iou(
        voxel_grid,
        PART_COLORS,
        mask_img,
        final_params,
        mode='whole_on_whole'
    )


### 5. Mask-based optimization

In [ ]:
final_camera_params = {}

for view_name in ["front", "drone"]:
    im  = views[view_name]                 # ← FIX
    cam = kp_camera_params[view_name]

    final_camera_params[view_name] = launch_smart_aligner(
        voxel_grid,
        im,
        PART_COLORS,
        parts_for_alignment=["front_minarets", "back_minarets"],
        init_params=cam,
    )


### 6. Save camera parameters

In [ ]:
def to_json_safe(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, dict):
        return {k: to_json_safe(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_json_safe(v) for v in obj]
    return obj

# --- Config ---
project_root = Path.cwd()

out_dir = (
    project_root
    / "results_temp"
    / "2.Perspective_Camera_Estimation"
)
out_dir.mkdir(parents=True, exist_ok=True)

# --- Camera parameter sets ---
camera_sets = {
    "init":  init_camera_params,
    "kp":    kp_camera_params,
    "final": final_camera_params
}

# --- Save ---
for tag, params in camera_sets.items():
    path = out_dir / f"{monument}_camera_params_{tag}.json"
    with open(path, "w") as f:
        json.dump(to_json_safe(params), f, indent=2)
    print(f"Saved: {path.resolve()}")
